In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterSampler
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
import re

# sklearn classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

%load_ext autoreload
%autoreload 2

Need a class that creates all the parameters for the classifier and random search. Needs to be reproducible.

Random search parameters:

* random seed
* number of iterations
* under/over sampling method
* imbalance ratio (for under/over sampling)
* scaling method (e.g. standard scaling, or min/max, etc.)
* features used
    * how to repeatably select the appropriate features? Maybe randomly shuffle?
    * use `random.choices` to select the features? Use `random.seed()` to set the seed.
* number of features used
* classifier used
* data set used

Dataset parameters (may be specific for dataset). Should probably store as a separate data card... For milling data set, parameters could be:

* label meta-data (df_labels)
* window size
* stride
* cut drop list

Classifier parameters:
* classifier
* parameter sampler seed


In [ ]:
# practice loading the datafram, selecting the column names, and randomly selecting n number of features (based on column names)

